In [1]:
from collections import defaultdict
import glob
import numpy as np
import os
import re

import pandas as pd
from tqdm import tqdm

In [40]:
# delta y

In [43]:
from tablebench.core.tabular_dataset import CachedDataset
cache_dir = "../tmp"

from tablebench.notebook_lib import UIDS_LIST, extract_name_from_uid
uids_list = UIDS_LIST


In [5]:
UIDS_LIST[-1]

'physionetICULOSgt47.0'

In [8]:
# list of (uid, split, n_split, ymean) tuples
results = []

uids_list = ['physionetICULOSgt47.0']
for uid in uids_list:
    print(f'processing uid {uid}')
    base_dir = os.path.join(cache_dir, uid)  
    if uid == 'physionetICULOSgt47.0':
        name = "physionet"
    else:
        name = extract_name_from_uid(uid)
    dset = CachedDataset(cache_dir=cache_dir, uid=uid, name=name)
    for split in ('train', 'id_test', 'ood_test', 'validation', 'ood_validation'):
        print(f'processing split {split}')
        yvals = list()
        split_files = glob.glob(os.path.join(base_dir, split, '*', '*.csv'))

        for f in tqdm(split_files):
            df = pd.read_csv(f)
            yvals.extend(df[dset.target].values.tolist())
        
        n_split = len(yvals)
        ybar_split = np.array(yvals).mean()
        results.append((uid, split, n_split, ybar_split))
        

processing uid acsfoodstampsdomain_split_varname_DIVISIONdomain_split_ood_value_06
processing split train


100%|█████████████████████████████████████████████████████████████████████████████████████| 158/158 [00:08<00:00, 19.54it/s]


processing split id_test


100%|███████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 22.45it/s]


processing split ood_test


...

100%|███████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 60.06it/s]


processing split validation


100%|███████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 60.48it/s]


processing split ood_validation


100%|█████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 65.32it/s]


In [9]:
df = pd.DataFrame(results, columns=['uid', 'split', 'n_split', 'ybar_split'])
df

,uid,split,n_split,ybar_split
0,physionetICULOSgt47.0,train,1122299,0.011850
1,physionetICULOSgt47.0,id_test,140288,0.011997
2,physionetICULOSgt47.0,ood_test,134402,0.075073
3,physionetICULOSgt47.0,validation,140287,0.011833
4,physionetICULOSgt47.0,ood_validation,14934,0.079282


In [10]:
df.pivot(index='uid', columns=['split'])

n_split                                              \
split                 id_test ood_test ood_validation    train validation   
uid                                                                         
physionetICULOSgt47.0  140288   134402          14934  1122299     140287   

                                                                             \
split                                               ood_test ood_validation   
uid                                                                           
acsfoodstampsdomain_split_varname_DIVISIONdomai...  0.219874       0.213589   
acsincomedomain_split_varname_DIVISIONdomain_sp...  0.397966       0.401897   
acspubcovdomain_split_varname_DISdomain_split_o...  0.636270       0.639850   
acsunemploymentdomain_split_varname_SCHLdomain_...  0.051512       0.050605   
anesdomain_split_varname_VCF0112domain_split_oo...  0.753968       0.750809   
brfss_blood_pressuredomain_split_varname_BMI5CA...  0.584293       0.583514   
brfss_diabetesdomain_split_varname_PRACE1domain...  0.174195       0.171252   
diabetes_readmissiondomain_split_varname_admiss...  0.493839       0.506356   
helocdomain_split_varname_ExternalRiskEstimateL...  0.568990       0.526658   
mimic_extract_los_3domain_split_varname_insuran...  0.455936       0.442249   
mimic_extract_mort_hospdomain_split_varname_ins...  0.123892       0.123588   
nhanes_leaddomain_split_varname_INDFMPIRBelowCu...  0.080324       0.072998   
physionetICULOSgt47.0                               0.075073       0.079282   

                                                                         
split                                                  train validation  
uid                                                                      
acsfoodstampsdomain_split_varname_DIVISIONdomai...  0.190058   0.187238  
acsincomedomain_split_varname_DIVISIONdomain_sp...  0.322643   0.323514  
acspubcovdomain_split_varname_DISdomain_split_o...  0.223905   0.222912  
acsunemploymentdomain_split_varname_SCHLdomain_...  0.034059   0.033409  
anesdomain_split_varname_VCF0112domain_split_oo...  0.790815   0.809615  
brfss_blood_pressuredomain_split_varname_BMI5CA...  0.401213   0.402203  
brfss_diabetesdomain_split_varname_PRACE1domain...  0.124719   0.125477  
diabetes_readmissiondomain_split_varname_admiss...  0.424230   0.421839  
helocdomain_split_varname_ExternalRiskEstimateL...  0.240991   0.197842  
mimic_extract_los_3domain_split_varname_insuran...  0.404216   0.375348  
mimic_extract_mort_hospdomain_split_varname_ins...  0.075464   0.071991  
nhanes_leaddomain_split_varname_INDFMPIRBelowCu...  0.025663   0.022358  
physionetICULOSgt47.0                               0.011850   0.011833

In [11]:
df

,uid,split,n_split,ybar_split
0,physionetICULOSgt47.0,train,1122299,0.011850
1,physionetICULOSgt47.0,id_test,140288,0.011997
2,physionetICULOSgt47.0,ood_test,134402,0.075073
3,physionetICULOSgt47.0,validation,140287,0.011833
4,physionetICULOSgt47.0,ood_validation,14934,0.079282


In [12]:
df.pivot(index='uid', columns=['split']).reset_index().to_csv("dataset_summaries_by_split.csv")

In [13]:
tmp = df.pivot(index='uid', columns=['split'])
ydiff_id_test_ood_test = tmp.loc[:,("ybar_split", "id_test")] - tmp.loc[:,("ybar_split", "ood_test")]
ydiff_id_val_id_test = tmp.loc[:,("ybar_split", "validation")] - tmp.loc[:,("ybar_split", "id_test")]

# (ydiff_id_test_ood_test**2)/abs(ydiff_id_val_id_test)
ydiff_id_test_ood_test**2

# ydiff_id_val_id_test

uid
physionetICULOSgt47.0    0.003979
dtype: float64

# otdd

In [ ]:
from otdd.pytorch.distance import DatasetDistance

In [ ]:
uid = 'helocdomain_split_varname_ExternalRiskEstimateLowdomain_split_ood_value_0'
base_dir = os.path.join(cache_dir, uid)  
name = extract_name_from_uid(uid)
dset = CachedDataset(cache_dir=cache_dir, uid=uid, name=name)

In [ ]:
import torch

def df_to_loader(df, target):
    x = df.drop(columns=target)
    y = df[target]
    x = torch.from_numpy(x.values)
    y = torch.from_numpy(y.values.astype(int))
    tensor_dataset = TensorDataset(x,y)
    loader = torch.utils.data.DataLoader(tensor_dataset, batch_size=64)
    return loader

In [ ]:
def read_split_data(base_dir):
    
    split_data = {}
    
    
    # Read the test datasets
    for split in ("ood_test", "id_test"):
        print(split)
        df_list = []
        split_files = glob.glob(os.path.join(base_dir, split, '*', '*.csv'))

        for f in tqdm(split_files):
            df = pd.read_csv(f)
            df_list.append(df)

        split_data[split] = pd.concat(df_list)
    return split_data

In [ ]:
results = []

for uid in uids_list:
    print(f"processing {uid}")
    base_dir = os.path.join(cache_dir, uid)  
    
    if uid == 'physionetICULOSgt47.0':
        name = "physionet"
    else:
        name = extract_name_from_uid(uid)
        
    dset = CachedDataset(cache_dir=cache_dir, uid=uid, name=name)
    
    split_data = read_split_data(base_dir)
    
    # Compare 
    assert np.all(split_data["id_test"].dtypes == split_data["ood_test"].dtypes)
    id_loader =  df_to_loader(split_data["id_test"], target=dset.target)
    ood_loader = df_to_loader(split_data["ood_test"], target=dset.target)
#     try:
    dist = DatasetDistance(id_loader, ood_loader,
                           # Note: this fails with 'exact' for some datasets.
                       inner_ot_method = 'gaussian_approx',
                       debiased_loss = True,
                       p = 2, entreg = 1e-1,
                       device='cpu')
    print(f"computing distance for {uid}")
    d = dist.distance(maxsamples = 10000)
    print(f'OTDD(src,tgt)={d}')
    results.append((uid, d))
#     except Exception as e:
#         print(e)
#         continue
    


In [ ]:

otdd_results_df = pd.DataFrame([(x[0], x[1].detach().cpu().numpy().item()) for x in results], columns=['uid', 'otdd'])
otdd_results_df.to_csv("otdd_results", index=False)
otdd_results_df
